In [ ]:
library(smoof)
library(ggplot2)
library(dplyr)
library(GA)

In [ ]:
rosenbrock_2d <- makeRosenbrockFunction(dimensions = 2)
rosenbrock_10d <- makeRosenbrockFunction(dimensions = 10)
rosenbrock_20d <- makeRosenbrockFunction(dimensions = 20)

rastrigin_2d <- makeRastriginFunction(dimensions = 2)
rastrigin_10d <- makeRastriginFunction(dimensions = 10)
rastrigin_20d <- makeRastriginFunction(dimensions = 20)

NO_STARTS <- 50

get_domain <- function(fun) {
  list(
    lower = getLowerBoxConstraints(fun),
    upper = getUpperBoxConstraints(fun)
  )
}

domains <- list(
  rosenbrock_2d = get_domain(rosenbrock_2d),
  rosenbrock_10d = get_domain(rosenbrock_10d),
  rosenbrock_20d = get_domain(rosenbrock_20d),
  rastrigin_2d = get_domain(rastrigin_2d),
  rastrigin_10d = get_domain(rastrigin_10d),
  rastrigin_20d = get_domain(rastrigin_20d)
)

print(domains)

In [ ]:
ms_run <- function(fun, domain, starts) {
  total_evals <- 0
  best_value <- Inf
  
  for (i in 1:starts) {
    x0 <- runif(length(domain$lower), domain$lower, domain$upper)
    
    result <- optim(x0, fun, method = "L-BFGS-B", lower = domain$lower, upper = domain$upper)
    
    total_evals <- total_evals + result$counts[["function"]]
    if (result$value < best_value) best_value <- result$value
  }
  
  list(best = best_value, evals = total_evals)
}

In [ ]:
run_experiments <- function(fun, domain, algorithm, budget, starts = NO_STARTS) {
  if (algorithm == "MS") {
    replicate(50, ms_run(fun, domain, starts)$best)
  } else if (algorithm == "GA") {
    replicate(50, ga_run(fun, domain, budget))
  }
}

ga_run <- function(fun, domain, budget) {
  ga_result <- ga(
    type = "real-valued",
    fitness = function(x) -fun(x),
    lower = domain$lower,
    upper = domain$upper,
    popSize = 50,
    maxiter = budget,
    run = 10,             # Stop if the best solution doesn't improve for 10 generations
    monitor = FALSE
  )
  
  return(-ga_result@fitnessValue)  # GA maximizes, so return the negative value
}

experiments <- list(
  rosenbrock_2d = list(fun = rosenbrock_2d, domain = domains$rosenbrock_2d),
  rosenbrock_10d = list(fun = rosenbrock_10d, domain = domains$rosenbrock_10d),
  rosenbrock_20d = list(fun = rosenbrock_20d, domain = domains$rosenbrock_20d),
  rastrigin_2d = list(fun = rastrigin_2d, domain = domains$rastrigin_2d),
  rastrigin_10d = list(fun = rastrigin_10d, domain = domains$rastrigin_10d),
  rastrigin_20d = list(fun = rastrigin_20d, domain = domains$rastrigin_20d)
)

ms_results <- lapply(experiments, function(exp) {
  ms_run(exp$fun, exp$domain, starts = NO_STARTS)
})

budget <- mean(sapply(ms_results, function(res) res$evals))

ga_results <- lapply(seq_along(experiments), function(i) {
  run_experiments(experiments[[i]]$fun, experiments[[i]]$domain, "GA", budget = budget)
})

ms_best_values <- lapply(ms_results, function(res) rep(res$best, 50))

results <- data.frame(
  Function = rep(names(experiments), each = 50),
  Algorithm = rep(c("MS", "GA"), each = 50 * length(experiments)),
  Value = c(unlist(ms_best_values), unlist(ga_results))
)

print(results)

In [ ]:
# res <- results %>% group_by(Function) %>% arrange(desc(Value)) %>% slice(-1:-5)
res <- results

In [ ]:
ggplot(res, aes(x = Function, y = Value, fill = Algorithm)) +
  geom_boxplot() +
  facet_wrap(~Function, scales = "free") +
  labs(title = "Comparison of PRS and MS on Rosenbrock and Rastrigin Functions",
       y = "Minimum Function Value",
       x = "Function and Dimension") +
  theme_minimal()

In [ ]:
statistical_analysis <- function(results) {
  results %>%
    group_by(Function, Algorithm) %>%
    summarise(Mean = mean(Value), SD = sd(Value)) %>%
    ungroup()
}

t_tests <- lapply(names(experiments), function(fun) {
  ms_values <- results %>% filter(Function == fun & Algorithm == "MS") %>% pull(Value)
  ga_values <- results %>% filter(Function == fun & Algorithm == "GA") %>% pull(Value)
  t.test(ms_values, ga_values)
})

t_tests